## Plots of SST for S-MODE region


In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import cftime

import functions  # requires functions.py from this directory
import cartopy.crs as ccrs                   # import projections
import cartopy
import gsw

# For great circle distance between two points, use gsw.geostrophy.distance(lon, lat, p=0, axis=-1)


In [2]:
# %matplotlib inline
%matplotlib qt5
plt.rcParams['figure.figsize'] = (5,4)
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 400
plt.close('all')

__figdir__ = '../plots/' + 'SMODE_'
savefig_args = {'bbox_inches':'tight', 'pad_inches':0.2}
plotfiletype='png'

In [3]:
savefig = True
zoom = False
if zoom:
    xmin, xmax = (-126,-122)
    ymin, ymax = (36,39)
else:
    xmin, xmax = (-127,-121)
    ymin, ymax = (35, 40)

    

In [4]:
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPB/AVHRR_METOPB_20210930T060000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20210930T065001Z.nc'3 
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20210929T213000Z.nc'  #This one is good

#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211016T095000Z.nc' # good one
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/AVHRR_METOPA/AVHRR_METOPA_20211019T031000Z.nc#bytes' #Really great!
# url = 'http://smode.whoi.edu:8080/thredds/fileServer/satellite/VIIRS_NRT/VIIRS_NRT_20211019T103001Z.nc#mode=bytes' #Not awesome, but coincident with B200 flight on 10/19
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/MODIS_Terra/MODIS_Terra_20211011T233459Z.nc'
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211022T111000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPA/AVHRR_METOPA_20211022T161000Z.nc' # good one!

# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211025T050000Z.nc' #not good, but looking for front...
# url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/VIIRS_NRT/VIIRS_NRT_20211025T102000Z.nc'
#url = 'http://smode.whoi.edu:8080/thredds/dodsC/satellite/AVHRR_METOPC/AVHRR_METOPC_20211025T190000Z.nc'

# 2022 IOP
url = 'http://smode.whoi.edu:8080/thredds/dodsC/IOP1_2022/satellite/VIIRS_NRT/VIIRS_NRT_20220921T101000Z.nc'; V = [14, 20]; zoom = 1 #


ds = xr.open_dataset(url)

In [5]:
ds

<xarray.Dataset>
Dimensions:                  (time: 1, nj: 980, ni: 1165)
Coordinates:
  * time                     (time) datetime64[ns] 2022-09-21T10:10:00
    lat                      (nj, ni) float32 ...
    lon                      (nj, ni) float32 ...
Dimensions without coordinates: nj, ni
Data variables:
    sea_surface_temperature  (time, nj, ni) float32 ...
    quality_level            (time, nj, ni) float32 ...
Attributes: (12/64)
    geospatial_bounds:                        POLYGON((-144.779  25.720, -143...
    geospatial_first_scanline_first_fov_lat:  60.24225
    geospatial_first_scanline_first_fov_lon:  -143.9065
    geospatial_first_scanline_last_fov_lat:   52.68042
    geospatial_first_scanline_last_fov_lon:   -94.25177
    geospatial_last_scanline_first_fov_lat:   25.719788
    ...                                       ...
    Conventions:                              CF-1.0
    History:                                  Translated to CF-1.0 Convention...
    geospatial_lat_min:                       32.38153076171875
    geospatial_lat_max:                       40.533203125
    geospatial_lon_min:                       -131.20941162109375
    geospatial_lon_max:                       -118.13836669921875

In [6]:
fig = plt.figure()
ax = plt.axes(projection = ccrs.PlateCarree(central_longitude=200))  # Orthographic
extent = [xmin, xmax, ymin, ymax]
ax.set_extent(extent, crs=ccrs.PlateCarree())
'''daystr=ds.time.dt.day.astype(str).values[0]
monstr=ds.time.dt.month.astype(str).values[0]
yrstr=ds.time.dt.year.astype(str).values[0]
day_str = monstr+'-'+daystr+'-'+yrstr
'''
day_str=ds.time.dt.strftime("%a, %b %d %H:%M").values[0]
day_str2=ds.time.dt.strftime("%m-%d-%Y").values[0]

ax.set_title('SST, ' + day_str, size = 10.)

#plt.set_cmap(cmap=plt.get_cmap('nipy_spectral'))
plt.set_cmap(cmap=plt.get_cmap('turbo'))
gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.5, linestyle='--')
gl.top_labels = False
gl.ylabels_right = False
#gl.xlocator = matplotlib.ticker.MaxNLocator(10)
#gl.xlocator = matplotlib.ticker.AutoLocator
# gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(0, 360 ,30))

ax.coastlines()
ax.add_feature(cartopy.feature.LAND, zorder=3, facecolor=[.6,.6,.6], edgecolor='black')
# cs = ax.contourf(ds.lon,ds.lat,np.squeeze(ds.sea_surface_temperature)-273.15, levels, extend='both', transform=ccrs.PlateCarree())
cs = ax.pcolormesh(ds.lon,ds.lat,np.squeeze(ds.sea_surface_temperature)-273.15, vmin=V[0], vmax=V[-1], transform=ccrs.PlateCarree())
# cb = plt.colorbar(cs,ax=ax,shrink=.8,pad=.05)
cb = plt.colorbar(cs,fraction = 0.022,extend='both')
cb.set_label('SST [$\circ$C]',fontsize = 10)
functions.plot_ops_area(ax,transform=ccrs.PlateCarree(),color='w')

# Add a 10 km scale bar
km_per_deg_lat=gsw.geostrophy.distance((125,125), (37,38))/1000
deg_lat_equal_10km=10/km_per_deg_lat
x0 = -125
y0 = 37.75
ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km]),transform=ccrs.PlateCarree(),color='k')
ax.text(x0+2/60, y0-.5/60, '10 km', fontsize=6,transform=ccrs.PlateCarree())

if savefig:
    plt.savefig(__figdir__+'SST_' + day_str2 + '.' +plotfiletype,**savefig_args)


/home/jtomf/miniconda3/lib/python3.9/site-packages/cartopy/mpl/gridliner.py:343: UserWarning: The .ylabels_right attribute is deprecated. Please use .right_labels to toggle visibility instead.
  warnings.warn('The .ylabels_right attribute is deprecated. Please '


{'transform': <cartopy.crs.PlateCarree object at 0x7f2dd71f4e50>, 'color': 'w'}


/tmp/ipykernel_17182/2001732645.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ax.plot(x0+np.asarray([0, 0]),y0+np.asarray([0.,deg_lat_equal_10km]),transform=ccrs.PlateCarree(),color='k')


In [7]:
ax.axes.get_extent()

(32.999999999999986, 39.000000000000014, 35.0, 40.0)

In [8]:
'''
1062	2021-10-25 14:23:10	37.47218	-125.03579
1072	2021-10-25 14:32:49	37.43843	-125.24681
1073	2021-10-25 14:20:56	37.36932	-125.20657
1074	2021-10-25 14:23:25	37.44937	-125.06652
1075	2021-10-25 14:22:42	37.39716	-125.02382

'''
xSD = [-125.03579, -125.24681, -125.20657, -125.06652, -125.02382]
ySD = [37.47218, 37.43843, 37.36932, 37.44937, 37.39716]
ax.plot(xSD,ySD,'ko',markersize=3,transform=ccrs.PlateCarree())
# ax.plot(xSD,ySD,'wo',transform=ccrs.PlateCarree())


In [9]:
if savefig:
    plt.savefig(__figdir__+'SST_' + day_str2 + '.' +plotfiletype,**savefig_args)
